<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data</a></span></li><li><span><a href="#OLS" data-toc-modified-id="OLS-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>OLS</a></span><ul class="toc-item"><li><span><a href="#LM.jl" data-toc-modified-id="LM.jl-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LM.jl</a></span></li><li><span><a href="#GMM" data-toc-modified-id="GMM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>GMM</a></span></li></ul></li><li><span><a href="#Logit" data-toc-modified-id="Logit-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Logit</a></span><ul class="toc-item"><li><span><a href="#GLM.jl" data-toc-modified-id="GLM.jl-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>GLM.jl</a></span></li><li><span><a href="#GMM" data-toc-modified-id="GMM-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>GMM</a></span></li></ul></li><li><span><a href="#Multinomial-Logit-Model" data-toc-modified-id="Multinomial-Logit-Model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Multinomial Logit Model</a></span><ul class="toc-item"><li><span><a href="#for-loop" data-toc-modified-id="for-loop-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>for loop</a></span></li><li><span><a href="#matrix-version" data-toc-modified-id="matrix-version-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>matrix version</a></span></li><li><span><a href="#with-Hessian" data-toc-modified-id="with-Hessian-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>with Hessian</a></span></li></ul></li><li><span><a href="#Multinomial-Logit-Model-with-Alternative-specific-Covariates-$Z$" data-toc-modified-id="Multinomial-Logit-Model-with-Alternative-specific-Covariates-$Z$-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Multinomial Logit Model with Alternative-specific Covariates $Z$</a></span></li><li><span><a href="#Nested-Logit" data-toc-modified-id="Nested-Logit-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Nested Logit</a></span></li><li><span><a href="#multinomial-logit-(with-alternative-specific-covariates-Z)" data-toc-modified-id="multinomial-logit-(with-alternative-specific-covariates-Z)-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>multinomial logit (with alternative-specific covariates Z)</a></span></li><li><span><a href="#mixed-logit" data-toc-modified-id="mixed-logit-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>mixed logit</a></span></li><li><span><a href="#test-DM" data-toc-modified-id="test-DM-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>test DM</a></span></li></ul></div>

In [1]:
;pwd

/c/Users/jhu/Documents/GitHub/fall-2020/ProblemSets


In [2]:
ENV["COLUMNS"] = 1000

1000

In [3]:
using Random
using LinearAlgebra
using Statistics
using Optim
using DataFrames
using CSV
using HTTP
using GLM
using FreqTables
using ForwardDiff 
using LineSearches
using Distributions

# Data

In [4]:
url = "https://raw.githubusercontent.com/OU-PhD-Econometrics/fall-2020/master/ProblemSets/PS1-julia-intro/nlsw88.csv"
df = CSV.read(HTTP.get(url).body)
X = [ones(size(df,1),1) df.age df.race.==1 df.collgrad.==1]
y = df.married.==1;

In [5]:
size(df),size(X), size(y), X[1,:],y[1:4]

((2246, 17), (2246, 4), (2246,), [1.0, 37.0, 0.0, 0.0], Bool[0, 0, 0, 1])

In [6]:
df[1:4,:]

,idcode,age,race,married,never_married,grade,collgrad,south,smsa,c_city,industry,occupation,union,wage,hours,ttl_exp,tenure
,Int64,Int64,Int64,Int64,Int64,Int64?,Int64,Int64,Int64,Int64,Int64?,Int64?,Int64?,Float64,Int64?,Float64,Float64?
1,1,37,2,0,0,12,0,0,1,0,5,6,1,11.7391,48,10.3333,5.33333
2,2,37,2,0,0,12,0,0,1,1,4,5,1,6.40096,40,13.6218,5.25
3,3,42,2,0,1,12,0,0,1,1,4,3,missing,5.01672,40,17.7308,1.25
4,4,43,1,1,0,17,1,0,1,0,11,13,1,9.03381,42,13.2115,1.75


# OLS

\begin{align*}
    \min_\beta \sum_i\left(y_i - 
    X_i\beta\right)^2
\end{align*}

In [7]:
function ols(β, X, y)
    ssr = (y.-X*β)'*(y.-X*β)
    return ssr
end

ols (generic function with 1 method)

In [8]:
β_ols = optimize(β -> ols(β, X, y), rand(size(X,2)), LBFGS(), 
        Optim.Options(g_tol=1e-6, iterations=100_000, show_trace=false))

println(β_ols.minimizer) 

[0.661350951535174, -0.004625908044343551, 0.2259504322824406, -0.012184197718293887]


In [9]:
β_ols = inv(X'*X)*X'*y
println(β_ols)

[0.6613509511877042, -0.004625908035375747, 0.22595043227530237, -0.01218419772065558]


## LM.jl

In [10]:
df.white = df.race.==1
bols_lm = lm(@formula(married ~ age + white + collgrad), df)
println(bols_lm)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

married ~ 1 + age + white + collgrad

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.661351    0.127491     5.19    <1e-6    0.411338   0.911364
age          -0.00462591  0.00324314  -1.43    0.1539  -0.0109858  0.00173397
white         0.22595     0.0223827   10.09    <1e-22   0.182057   0.269843
collgrad     -0.0121842   0.0233565   -0.52    0.6020  -0.0579868  0.0336184
─────────────────────────────────────────────────────────────────────────────


## GMM

In [11]:
function ols_gmm(α, X, y)    
    g = y .- X*α
    J = g'*I*g
    return J
end

ols_gmm (generic function with 1 method)

In [12]:
α̂_optim = optimize(a -> ols_gmm(a, X, y), rand(size(X,2)), LBFGS(), Optim.Options(g_tol=1e-8, iterations=100_000))
println(α̂_optim.minimizer)

[0.6613509519821575, -0.004625908055546829, 0.2259504322771398, -0.012184197726113821]


# Logit

In [13]:
function logit(β, X, y)

    P = exp.(X*β)./(1 .+ exp.(X*β))

    loglike = -sum( (y.==1).*log.(P) .+ (y.==0).*log.(1 .- P) )

    return loglike
end

logit (generic function with 1 method)

In [14]:
β_optim = optimize(β -> logit(β, X, y), rand(size(X,2)), LBFGS(), 
           Optim.Options(g_tol=1e-6, iterations=100_000, show_trace=false))
println(β_optim.minimizer)

[0.7465534201381056, -0.02107775603173581, 0.9558065345399978, -0.05597400452588138]


## GLM.jl

In [15]:
β_glm = glm(@formula(married ~ age + white + collgrad), df, Binomial(), LogitLink())
println(β_glm)

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},LogitLink},GLM.DensePredChol{Float64,Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

married ~ 1 + age + white + collgrad

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      z  Pr(>|z|)   Lower 95%   Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.746554    0.578983    1.29    0.1973  -0.388231   1.88134
age          -0.0210778   0.0147497  -1.43    0.1530  -0.0499867  0.00783116
white         0.955807    0.0981953   9.73    <1e-21   0.763347   1.14827
collgrad     -0.055974    0.106218   -0.53    0.5982  -0.264157   0.152209
────────────────────────────────────────────────────────────────────────────


## GMM

In [16]:
function logit_gmm(β, X, y)
    P = exp.(X*β)./(1 .+ exp.(X*β))
    g = y .- P
    J = g'*I*g
    return J
end


logit_gmm (generic function with 1 method)

In [17]:
β_optim = optimize(β -> logit_gmm(β, X, y), rand(size(X,2)), LBFGS(), 
                   Optim.Options(g_tol=1e-8, iterations=100_000))

println(β_optim.minimizer)

[0.7504022858104027, -0.021198601931836897, 0.9554883097159053, -0.052070109160042205]


# Multinomial Logit Model

In [18]:
freqtable(df, :occupation)

14-element Named Array{Int64,1}
occupation  │ 
────────────┼────
1           │ 317
2           │ 264
3           │ 726
4           │ 102
5           │  53
6           │ 246
7           │  28
8           │ 286
9           │   1
10          │   9
11          │  16
12          │   2
13          │ 187
missing     │   9

In [19]:
df = dropmissing(df, :occupation)
df[df.occupation.==8 ,:occupation] .= 7
df[df.occupation.==9 ,:occupation] .= 7
df[df.occupation.==10,:occupation] .= 7
df[df.occupation.==11,:occupation] .= 7
df[df.occupation.==12,:occupation] .= 7
df[df.occupation.==13,:occupation] .= 7
freqtable(df, :occupation) # problem solved

7-element Named Array{Int64,1}
occupation  │ 
────────────┼────
1           │ 317
2           │ 264
3           │ 726
4           │ 102
5           │  53
6           │ 246
7           │ 529

In [20]:
X = [ones(size(df,1),1) df.age df.race.==1 df.collgrad.==1]
y = df.occupation

size(X), size(y), X[1,:],y[1:4]

((2237, 4), (2237,), [1.0, 37.0, 0.0, 0.0], [6, 5, 3, 7])

## for loop

In [21]:
function mlogit(alpha, X, y)
    
    K = size(X,2)
    J = length(unique(y))
    N = length(y)
    bigY = zeros(N,J)
    for j=1:J
        bigY[:,j] = y.==j
    end
    bigAlpha = [reshape(alpha,K,J-1) zeros(K)]
    
    num = zeros(N,J)
    dem = zeros(N)
    for j=1:J
        num[:,j] = exp.(X*bigAlpha[:,j])
        dem .+= num[:,j]
    end
    
    P = num./repeat(dem,1,J)
    
    loglike = -sum( bigY.*log.(P) )
    
    return loglike
end

mlogit (generic function with 1 method)

In [22]:
alpha_zero = zeros(6*size(X,2))
alpha_rand = rand(6*size(X,2))
alpha_true = [0.1910213
 -0.0335262
  0.5963968
  0.4165052
 -0.1698368
 -0.0359784
  1.30684
 -0.430997
  0.6894727
 -0.0104578
  0.5231634
 -1.492475
 -2.26748
 -0.0053001
  1.391402
 -0.9849661
 -1.398468
 -0.0142969
 -0.0176531
 -1.495123
  0.2454891
 -0.0067267
 -0.5382892
 -3.78975]

alpha_start = alpha_true.*rand(size(alpha_true))
println(size(alpha_true))

(24,)


In [23]:
alpha_hat_optim = optimize(a -> mlogit(a, X, y), alpha_start, LBFGS(), 
               Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false, show_every=500))
alpha_hat_mle = alpha_hat_optim.minimizer
println(alpha_hat_mle)

[0.1910399136067419, -0.03352669726199581, 0.5963964533422411, 0.41650682254074517, -0.16981704681365417, -0.03597891914556963, 1.3068417348600714, -0.4309976813533497, 0.6894687649341755, -0.010457750613984178, 0.523163541217355, -1.4924725134974826, -2.2675054525730656, -0.0052995443084240534, 1.3914081829484672, -0.9849694461929859, -1.3985573317180762, -0.014294755144515744, -0.017649054627850436, -1.4951229891987001, 0.24546631806612798, -0.00672617103251503, -0.5382873987598918, -3.7897820064356496]


## matrix version

In [24]:
function mlogit2(β, X, y)
        
    K = size(X, 2)
    J = length(unique(y))
    N = length(y)
    Y = zeros(N, J)
    for j = 1:J
        Y[:,j] = y .== j
    end
    Β = [reshape(β, K, J - 1) zeros(K)]
        
    V = X * Β
    num = exp.(V)
    dem = sum.(eachrow(num))
    P = num ./ dem
        
    loglike = -sum(Y .* log.(P))
        
    return loglike
end

mlogit2 (generic function with 1 method)

In [25]:
β_hat_optim = optimize(β -> mlogit2(β, X, y), alpha_start, LBFGS(), 
               Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false))
β_hat_mle = β_hat_optim.minimizer
println(β_hat_mle)

[0.1909769941470058, -0.03352504690436137, 0.5963943649961156, 0.4165056616481326, -0.16989694116948678, -0.035976860508479515, 1.3068399975440717, -0.4309971146157321, 0.6894328442008153, -0.010456843135641025, 0.5231634628995318, -1.4924727790455, -2.26757505605333, -0.005297765330805531, 1.3914062264536815, -0.9849650420044826, -1.3985809630498485, -0.014294134181603572, -0.017651634385692297, -1.4951172024192974, 0.24544082506821177, -0.006725500433777165, -0.5382895688273234, -3.789769371059199]


## with Hessian

In [26]:
function mlogit_for_h(alpha, X, y)
    
    K = size(X,2)
    J = length(unique(y))
    N = length(y)
    bigY = zeros(N,J)
    for j=1:J
        bigY[:,j] = y.==j
    end
    bigAlpha = [reshape(alpha,K,J-1) zeros(K)]
    
    T = promote_type(eltype(X),eltype(alpha)) # this line is new
    num   = zeros(T,N,J)                      # this line is new
    dem   = zeros(T,N)                        # this line is new
    for j=1:J
        num[:,j] = exp.(X*bigAlpha[:,j])
        dem .+= num[:,j]
    end
    
    P = num./repeat(dem,1,J)
    
    loglike = -sum( bigY.*log.(P) )
    
    return loglike
end

mlogit_for_h (generic function with 1 method)

In [27]:
# declare that the objective function is twice differentiable
td = TwiceDifferentiable(b -> mlogit_for_h(b, X, y), alpha_start; autodiff = :forward)
# run the optimizer
alpha_hat_optim_ad = optimize(td, alpha_zero, LBFGS(), Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false, show_every=50))
alpha_hat_mle_ad = alpha_hat_optim_ad.minimizer

# evaluate the Hessian at the estimates
H  = Optim.hessian!(td, alpha_hat_mle_ad)
alpha_hat_mle_ad_se = sqrt.(diag(inv(H)))

24-element Array{Float64,1}:
 0.9250426228059639
 0.02359600092699672
 0.16643185567818417
 0.14539833286543336
 0.9877635472143963
 0.025041034197731793
 0.2088348450186434
 0.16617533842935772
 0.7574422123655382
 0.019257796020397386
 0.13049479985889972
 0.14760682990849738
 1.414571804288081
 0.03552992911156632
 0.3224823529296227
 0.2716328320671566
 1.8591571577880135
 0.04742956036488801
 0.30354607050491805
 0.4434233597569664
 1.0264137574720964
 0.026162920649010726
 0.16340581853775987
 0.5881855599964643

In [28]:
DataFrame([alpha_hat_mle_ad alpha_hat_mle_ad_se])

,x1,x2
,Float64,Float64
1,0.191049,0.925043
2,-0.0335269,0.023596
3,0.596397,0.166432
4,0.416504,0.145398
5,-0.169818,0.987764
6,-0.0359789,0.025041
7,1.30684,0.208835
8,-0.430997,0.166175
9,0.689487,0.757442


# Multinomial Logit Model with Alternative-specific Covariates $Z$

In [29]:
url = "https://raw.githubusercontent.com/OU-PhD-Econometrics/fall-2020/master/ProblemSets/PS3-gev/nlsw88w.csv"
df = CSV.read(HTTP.get(url).body)
X = [df.age df.white df.collgrad]
Z = hcat(df.elnwage1, df.elnwage2, df.elnwage3, df.elnwage4, 
         df.elnwage5, df.elnwage6, df.elnwage7, df.elnwage8)
y = df.occupation;

In [30]:
size(df), size(X), size(y), X[1,:],y[1:4]

((2237, 13), (2237, 3), (2237,), [37, 0, 0], [6, 5, 3, 8])

In [31]:
df[1:2,:]

,idcode,age,collgrad,occupation,white,elnwage1,elnwage2,elnwage3,elnwage4,elnwage5,elnwage6,elnwage7,elnwage8
,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,37,0,6,0,2.09645,2.06278,1.83843,1.71633,1.82249,1.5234,1.2562,1.48748
2,2,37,0,5,0,2.16366,2.10306,1.82444,1.90102,1.81032,1.65941,1.43933,1.52887


\begin{align}
    P_{ij} &= \begin{cases} \frac{\exp\left(X_{i}\beta_j + \gamma(Z_{ij}-Z_{iJ})\right)}{1+\sum_{k=1}^{J-1}\exp\left(X_{i}\beta_k + \gamma(Z_{ik}-Z_{iJ})\right)} ,& j=1,\ldots,J-1\\
     \frac{1}{1+\sum_{k=1}^{J-1}\exp\left(X_{i}\beta_k + \gamma(Z_{ik}-Z_{iJ})\right)} ,& j=J
     \end{cases}
\end{align}

Normalize $\beta_J = 0$

In [32]:
function mlogit_with_Z(theta, X, Z, y)
    
    alpha = theta[1:end-1]
    gamma = theta[end]
    K = size(X,2)
    J = length(unique(y))
    N = length(y)
    bigY = zeros(N,J)
    for j=1:J
        bigY[:,j] = y.==j
    end
    bigAlpha = [reshape(alpha,K,J-1) zeros(K)]
    
    T = promote_type(eltype(X),eltype(theta))
    num   = zeros(T,N,J)
    dem   = zeros(T,N)
    for j=1:J
        num[:,j] = exp.(X*bigAlpha[:,j] .+ (Z[:,j] .- Z[:,J])*gamma)
        dem .+= num[:,j]
    end
    
    P = num./repeat(dem,1,J)
    
    loglike = -sum( bigY.*log.(P) )
    
    return loglike
end

mlogit_with_Z (generic function with 1 method)

In [33]:
startvals = [2*rand(7*size(X,2)).-1; .1]
td = TwiceDifferentiable(theta -> mlogit_with_Z(theta, X, Z, y), startvals; autodiff = :forward)
# run the optimizer
theta_hat_optim_ad = optimize(td, startvals, LBFGS(), Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false, show_every=50))
theta_hat_mle_ad = theta_hat_optim_ad.minimizer
# evaluate the Hessian at the estimates
H  = Optim.hessian!(td, theta_hat_mle_ad)
theta_hat_mle_ad_se = sqrt.(diag(inv(H)))
println([theta_hat_mle_ad theta_hat_mle_ad_se]) # these standard errors match Stata

[0.05570773127400288 0.009006608489811288; 0.08342354076033426 0.2390042493528875; -2.3448875410640393 0.29596455764141905; 0.04500088794573018 0.009279355672487905; 0.73657144530772 0.27918938270715554; -3.1532456645091793 0.2831673545000175; 0.09264615884886869 0.007682155703471486; -0.08418117998942762 0.23409181086359482; -4.2732804665511175 0.3124755780343901; 0.02390346651324 0.01005444731711928; 0.7230641744342566 0.3690323224481228; -3.7493918111151627 0.4071233609138687; 0.03608743252511334 0.009565098313695144; -0.6437694742681178 0.3572090302061202; -4.27969124108795 0.49832218529156386; 0.08531104922229273 0.0076424492278347285; -1.1714344726398032 0.26927456463084404; -6.6786729574990416 0.7230348055995423; 0.08662030223739378 0.008009171976668158; -0.7978815130206782 0.26677279532381165; -4.9691336564608175 0.36647766722664404; -0.09419279477250975 0.3790762787309687]


In [34]:
function mlogit_with_Z2(θ, X, Z, y)


    β = θ[1:end - 1]
    γ  = θ[end]

    Y = zeros(N, J)
    for j = 1:J
        Y[:,j] = y .== j
    end
    Β = [reshape(β, K, J - 1) zeros(K)]
    
    V = X * Β .+  (Z .- Z[:,J]) * γ    
    num = exp.(V)
    dem = sum.(eachrow(num))
    P = num ./ dem

    loglike = -sum(Y .* log.(P))
    
    return loglike
end

mlogit_with_Z2 (generic function with 1 method)

In [35]:
K = size(X, 2)
J = length(unique(y))
N = length(y)

startvals = [2 * rand((J - 1) * K) .- 1; .1]
td = TwiceDifferentiable(θ -> mlogit_with_Z(θ, X, Z, y), startvals; autodiff=:forward)
# run the optimizer
theta_hat_optim_ad = optimize(td, startvals, LBFGS(), Optim.Options(g_tol=1e-5, iterations=100_000, show_trace=false, show_every=50))
theta_hat_mle_ad = theta_hat_optim_ad.minimizer
# evaluate the Hessian at the estimates
H  = Optim.hessian!(td, theta_hat_mle_ad)
theta_hat_mle_ad_se = sqrt.(diag(inv(H)))
println([theta_hat_mle_ad theta_hat_mle_ad_se]) # these standard errors match Stata
DataFrame([theta_hat_mle_ad ,theta_hat_mle_ad_se])

[0.05570755411539571 0.00900660124515387; 0.0834301064931775 0.23900409355238855; -2.3448858746990497 0.2959644276733717; 0.04500066934609496 0.009279349305237994; 0.7365795690277666 0.27918929862395014; -3.153243303707476 0.2831672123339646; 0.09264595895569203 0.00768214602408022; -0.0841735363991131 0.23409163065204563; -4.2732785044198405 0.3124754562212242; 0.02390329555774093 0.010054439201982398; 0.7230708216291999 0.36903219549209254; -3.749391758037522 0.40712337360742656; 0.036087164451938325 0.009565094421201112; -0.643758197842855 0.35720899324194405; -4.279684727208941 0.4983214563667787; 0.08531084999978508 0.007642438700464445; -1.1714269247026452 0.26927438314431495; -6.678663537102998 0.7230332995937045; 0.08662010773872242 0.008009161575464855; -0.7978742557272328 0.2667726094601155; -4.969128908246748 0.3664774160746197; -0.09419244280638404 0.3790762798267904]


,x1,x2
,Float64,Float64
1,0.0557076,0.0090066
2,0.0834301,0.239004
3,-2.34489,0.295964
4,0.0450007,0.00927935
5,0.73658,0.279189
6,-3.15324,0.283167
7,0.092646,0.00768215
8,-0.0841735,0.234092
9,-4.27328,0.312475


# Nested Logit

Estimate a nested logit with the following nesting structure:  

* White collar occupations (indexed by WC)  
    1 Professional/Technical  
    2 Managers/Administrators  
    3 Sales  
* Blue collar occupations (indexed by BC)  
    4 Clerical/Unskilled  
    5 Craftsmen  
    6 Operatives  
    7 Transport  
* Other occupations (indexed by Other)  
    8 Other  

$\beta_{\text{Other}}$ is normalized to 0

\begin{align}
    P_{ij} &= \begin{cases} \frac{\exp\left(\frac{X_{i}\beta_{WC}+\gamma(Z_{ij}-Z_{iJ})}{\lambda_{WC}}\right)\left[\sum_{\ell\in WC}\exp\left(\frac{X_{i}\beta_{WC}+\gamma(Z_{i\ell}-Z_{iJ})}{\lambda_{WC}}\right)\right]^{\lambda_{WC}-1}}{1+\left[\sum_{k\in WC}\exp\left(\frac{X_{i}\beta_{WC}+\gamma(Z_{ik}-Z_{iJ})}{\lambda_{WC}}\right)\right]^{\lambda_{WC}}+\left[\sum_{m\in BC}\exp\left(\frac{X_{i}\beta_{BC}+\gamma(Z_{im}-Z_{iJ})}{\lambda_{BC}}\right)\right]^{\lambda_{BC}}} ,& j\in WC\\
    \frac{\exp\left(\frac{X_{i}\beta_{BC}+\gamma(Z_{ij}-Z_{iJ})}{\lambda_{BC}}\right)\left[\sum_{\ell\in BC}\exp\left(\frac{X_{i}\beta_{BC}+\gamma(Z_{i\ell}-Z_{iJ})}{\lambda_{BC}}\right)\right]^{\lambda_{BC}-1}}{1+\left[\sum_{k\in WC}\exp\left(\frac{X_{i}\beta_{WC}+\gamma(Z_{ik}-Z_{iJ})}{\lambda_{WC}}\right)\right]^{\lambda_{WC}}+\left[\sum_{m\in BC}\exp\left(\frac{X_{i}\beta_{BC}+\gamma(Z_{im}-Z_{iJ})}{\lambda_{BC}}\right)\right]^{\lambda_{BC}}} ,& j\in BC\\
     \frac{1}{1+\left[\sum_{k\in WC}\exp\left(\frac{X_{i}\beta_{WC}+\gamma(Z_{ik}-Z_{iJ})}{\lambda_{WC}}\right)\right]^{\lambda_{WC}}+\left[\sum_{m\in BC}\exp\left(\frac{X_{i}\beta_{BC}+\gamma(Z_{im}-Z_{iJ})}{\lambda_{BC}}\right)\right]^{\lambda_{BC}}} ,& j=J
     \end{cases}
\end{align}

In [36]:
function nested_logit_with_Z(theta, X, Z, y, nesting_structure)
    
    alpha = theta[1:end-3]
    lambda = theta[end-2:end-1]
    gamma = theta[end]
    K = size(X,2)
    J = length(unique(y))
    N = length(y)
    bigY = zeros(N,J)
    for j=1:J
        bigY[:,j] = y.==j
    end
    bigAlpha = [repeat(alpha[1:K],1,length(nesting_structure[1])) repeat(alpha[K+1:2K],1,length(nesting_structure[2])) zeros(K)]
    
    T = promote_type(eltype(X),eltype(theta))
    num   = zeros(T,N,J)
    lidx  = zeros(T,N,J)
    dem   = zeros(T,N)
    for j=1:J
        if j in nesting_structure[1]
            lidx[:,j] = exp.( (X*bigAlpha[:,j] .+ (Z[:,j] .- Z[:,J])*gamma)./lambda[1] )
        elseif j in nesting_structure[2]
            lidx[:,j] = exp.( (X*bigAlpha[:,j] .+ (Z[:,j] .- Z[:,J])*gamma)./lambda[2] )
        else
            lidx[:,j] = exp.(zeros(N))
        end
    end
    for j=1:J
        if j in nesting_structure[1]
            num[:,j] = lidx[:,j].*sum(lidx[:,nesting_structure[1][:]];dims=2).^(lambda[1]-1)
        elseif j in nesting_structure[2]
            num[:,j] = lidx[:,j].*sum(lidx[:,nesting_structure[2][:]];dims=2).^(lambda[2]-1)
        else
            num[:,j] = lidx[:,j]
        end
        dem .+= num[:,j]
    end
    
    P = num./repeat(dem,1,J)
    
    loglike = -sum( bigY.*log.(P) )
    
    return loglike
end

nested_logit_with_Z (generic function with 1 method)

In [37]:
nesting_structure = [[1 2 3], [4 5 6 7]]
startvals = [2*rand(2*size(X,2)).-1; 1; 1; .1]
size(startvals)

(9,)

In [38]:
td = TwiceDifferentiable(theta -> nested_logit_with_Z(theta, X, Z, y, nesting_structure), startvals; autodiff = :forward)

# run the optimizer
nlogit_theta_hat_optim_ad = optimize(td, startvals, LBFGS(), 
        Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false, show_every=50))
nlogit_theta_hat_mle_ad = nlogit_theta_hat_optim_ad.minimizer

# evaluate the Hessian at the estimates
#H  = Optim.hessian!(td, nlogit_theta_hat_mle_ad)
#nlogit_theta_hat_mle_ad_se = sqrt.(diag(inv(H)))

9-element Array{Float64,1}:
  0.09829427034738843
  0.10586649186567298
 -3.0829405550129723
  0.11167066369792215
 -0.7882728279263643
 -4.351956598892889
 -0.2841165381313727
 -0.24717966053110263
  0.6722654482132407

In [39]:
DataFrame([nlogit_theta_hat_mle_ad nlogit_theta_hat_mle_ad_se]) # these standard errors match Stata

LoadError: UndefVarError: nlogit_theta_hat_mle_ad_se not defined

# multinomial logit (with alternative-specific covariates Z) 

In [40]:
;pwd

/c/Users/jhu/Documents/GitHub/fall-2020/ProblemSets


In [41]:
include("PS4-mixture/lgwt.jl")

lgwt (generic function with 3 methods)

In [42]:
url = "https://raw.githubusercontent.com/OU-PhD-Econometrics/fall-2020/master/ProblemSets/PS4-mixture/nlsw88t.csv"
df = CSV.read(HTTP.get(url).body)
X = [df.age df.white df.collgrad]
Z = hcat(df.elnwage1, df.elnwage2, df.elnwage3, df.elnwage4, 
         df.elnwage5, df.elnwage6, df.elnwage7, df.elnwage8)
y = df.occ_code;

In [43]:
size(df),size(X), size(y), X[1,:],y[1:4]

((28365, 14), (28365, 3), (28365,), [18, 0, 0], [3, 6, 6, 6])

In [44]:
df[1:4,:]

,idcode,year,age,collgrad,occ_code,white,elnwage1,elnwage2,elnwage3,elnwage4,elnwage5,elnwage6,elnwage7,elnwage8
,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,70,18,0,3,0,1.54409,1.39779,1.5373,1.24634,1.32091,1.38799,1.05836,1.48378
2,1,71,19,0,6,0,1.57714,1.3839,1.54069,1.2108,1.30514,1.37949,1.11104,1.48257
3,1,72,20,0,6,0,1.62328,1.43399,1.56725,1.24377,1.36105,1.40231,1.15108,1.49813
4,1,73,21,0,6,0,1.66722,1.48198,1.5924,1.27541,1.41339,1.42391,1.18879,1.5135


In [45]:
function mlogit_with_Z(theta, X, Z, y)
        
        alpha = theta[1:end-1]
        gamma = theta[end]
        K = size(X,2)
        J = length(unique(y))
        N = length(y)
        bigY = zeros(N,J)
        for j=1:J
            bigY[:,j] = y.==j
        end
        bigAlpha = [reshape(alpha,K,J-1) zeros(K)]
        
        T = promote_type(eltype(X),eltype(theta))
        num   = zeros(T,N,J)
        dem   = zeros(T,N)
        for j=1:J
            num[:,j] = exp.(X*bigAlpha[:,j] .+ (Z[:,j] .- Z[:,J])*gamma)
            dem .+= num[:,j]
        end
        
        P = num./repeat(dem,1,J)
        
        loglike = -sum( bigY.*log.(P) )
        
        return loglike
    end

mlogit_with_Z (generic function with 1 method)

In [46]:
#startvals = [2*rand(7*size(X,2)).-1; .1]
startvals = [0.0403744
  0.2439942
 -1.57132
  0.0433254
  0.1468556
 -2.959103
  0.1020574
  0.7473086
 -4.12005
  0.0375628
  0.6884899
 -3.65577
  0.0204543
 -0.3584007
 -4.376929
  0.1074636
 -0.5263738
 -6.199197
  0.1168824
 -0.2870554
 -5.322248
  1.307477
]

22-element Array{Float64,1}:
  0.0403744
  0.2439942
 -1.57132
  0.0433254
  0.1468556
 -2.959103
  0.1020574
  0.7473086
 -4.12005
  0.0375628
  0.6884899
 -3.65577
  0.0204543
 -0.3584007
 -4.376929
  0.1074636
 -0.5263738
 -6.199197
  0.1168824
 -0.2870554
 -5.322248
  1.307477

In [47]:
td = TwiceDifferentiable(theta -> mlogit_with_Z(theta, X, Z, y), startvals; autodiff = :forward)
# run the optimizer
theta_hat_optim_ad = optimize(td, startvals, LBFGS(), 
                    Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false, show_every=50))

 * Status: success

 * Candidate solution
    Final objective value:     4.488317e+04

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 6.71e-09 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.08e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 2.67e-04 ≰ 1.0e-05

 * Work counters
    Seconds run:   15  (vs limit Inf)
    Iterations:    15
    f(x) calls:    49
    ∇f(x) calls:   49


In [48]:
theta_hat_mle_ad = theta_hat_optim_ad.minimizer
H  = Optim.hessian!(td, theta_hat_mle_ad)
theta_hat_mle_ad_se = sqrt.(diag(inv(H)))
DataFrame([theta_hat_mle_ad theta_hat_mle_ad_se]) # these standard errors match Stata

,x1,x2
,Float64,Float64
1,0.0403745,0.0028898
2,0.243995,0.0714891
3,-1.57132,0.0875003
4,0.0433255,0.00293729
5,0.146856,0.078683
6,-2.95911,0.0927874
7,0.102057,0.00254176
8,0.747309,0.0675232
9,-4.12005,0.0901064


# mixed logit 

In [49]:
# define distribution
d = Normal(0,1) # mean=0, standard deviation=1

# get quadrature nodes and weights for 7 grid points
nodes, weights = lgwt(7,-4,4)

# now compute the integral over the density and verify it's 1
sum(weights.*pdf.(d,nodes))

# now compute the expectation and verify it's 0
sum(weights.*nodes.*pdf.(d,nodes))

2.6454533008646308e-17

In [50]:
d = Normal(0,2)

nodes, weights = lgwt(7,-10,10)
sum(weights .* nodes.^2 .* pdf.(d,nodes))

nodes, weights = lgwt(10,-10,10)
sum(weights .* nodes.^2 .* pdf.(d,nodes))

4.038977384853661

In [51]:
b = 10
a = -10
c = Uniform(a,b)
draws = rand(c,1_000_000)

1000000-element Array{Float64,1}:
  4.070713908168839
 -3.852308047314703
  4.683207177063316
  8.526352865325872
  6.186739543836829
  0.1865782086046508
  3.1342593924678486
 -6.839183496257624
  4.448562033854326
 -1.7085196156614444
  0.5343517630544437
  4.723446681645118
  9.487440968202165
  ⋮
  8.291261860641637
  9.50520490339705
  7.228548558080554
  8.825946422805643
  4.557403397587958
 -2.5759613271378257
  3.419420293110562
  0.6801733516564479
 -3.2013875672136916
  9.765552765355523
 -2.46708171114272
  8.798004302515285

In [52]:
# estimate of variance
    (b-a)*mean(draws.^2 .* pdf.(d,draws)) |> println

4.003580702832545


In [53]:
# estimate of mean
    (b-a)*mean(draws .* pdf.(d,draws)) |> println

0.002088000105673679


In [54]:
# estimate of integral over support
    (b-a)*mean(pdf.(d,draws)) |> println

1.0001736020694412


In [55]:
# now repeat with only 1_000 draws
    draw2 = rand(c,1_000)
    (b-a)*mean(draws.^2 .* pdf.(d,draws)) |> println
    (b-a)*mean(draws .* pdf.(d,draws)) |> println
    (b-a)*mean(pdf.(d,draws)) |> println


4.003580702832545
0.002088000105673679
1.0001736020694412


In [56]:
function mixlogit_quad_with_Z(theta, X, Z, y, R)
    
    alpha = theta[1:end-2]
    gamma = theta[end-1]
    sigma = exp(theta[end]) # makes sure sigma is never 0 (which would make the Normal density undefined)
    K = size(X,2)
    J = length(unique(y))
    N = length(y)
    bigY = zeros(N,J)
    for j=1:J
        bigY[:,j] = y.==j
    end
    bigAlpha = [reshape(alpha,K,J-1) zeros(K)]
    
    # now write P as a function of variable of integration
    T = promote_type(eltype(X),eltype(theta))
    function like_int(T,N,J,X,Z,bigAlpha,gamma,sigma,R)
        nodes, weights = lgwt(R,-4*sigma,4*sigma)
        out = zeros(T,N)
        for r=1:R
            num   = zeros(T,N,J)
            dem   = zeros(T,N)
            P     = zeros(T,N,J)
            for j=1:J
                num[:,j] = exp.(X*bigAlpha[:,j] .+ (Z[:,j] .- Z[:,J])*nodes[r])
                dem .+= num[:,j]
            end
            P = num./repeat(dem,1,J)
            out .+= vec( weights[r]*( prod(P.^bigY; dims=2) )*pdf(Normal(gamma,sigma),nodes[r]) ) # why do we have to use vec here? because vectors in Julia are "flat" and the vec operator flattens a one-dimensional array.
        end
        return out
    end
    
    intlike = like_int(T,N,J,X,Z,bigAlpha,gamma,sigma,R)
    loglike = -sum(log.(intlike))
    return loglike
end

mixlogit_quad_with_Z (generic function with 1 method)

In [57]:
startvals = [
    0.0403744
  0.2439942
 -1.57132
  0.0433254
  0.1468556
 -2.959103
  0.1020574
  0.7473086
 -4.12005
  0.0375628
  0.6884899
 -3.65577
  0.0204543
 -0.3584007
 -4.376929
  0.1074636
 -0.5263738
 -6.199197
  0.1168824
 -0.2870554
 -5.322248
  1.307477
  2.0
]

23-element Array{Float64,1}:
  0.0403744
  0.2439942
 -1.57132
  0.0433254
  0.1468556
 -2.959103
  0.1020574
  0.7473086
 -4.12005
  0.0375628
  0.6884899
 -3.65577
  0.0204543
 -0.3584007
 -4.376929
  0.1074636
 -0.5263738
 -6.199197
  0.1168824
 -0.2870554
 -5.322248
  1.307477
  2.0

In [58]:
startvals = [
    -0.0493358
 -0.7359427
  1.161792
 -0.0127601
  0.1499798
 -2.438863
  0.0979789
  0.6508201
 -4.049595
  0.055819
  0.2773671
 -4.34042
  0.0203963
 -0.6880248
 -4.794188
  0.1288023
 -0.9974547
 -7.914539
  0.118583
 -0.7813687
 -5.609388
  4.50982
  2.2925
]

23-element Array{Float64,1}:
 -0.0493358
 -0.7359427
  1.161792
 -0.0127601
  0.1499798
 -2.438863
  0.0979789
  0.6508201
 -4.049595
  0.055819
  0.2773671
 -4.34042
  0.0203963
 -0.6880248
 -4.794188
  0.1288023
 -0.9974547
 -7.914539
  0.118583
 -0.7813687
 -5.609388
  4.50982
  2.2925

In [59]:
td = TwiceDifferentiable(theta -> mixlogit_quad_with_Z(theta, X, Z, y, 7), startvals; autodiff = :forward)
    # run the optimizer
theta_hat_mix_optim_ad = optimize(td, startvals, LBFGS(linesearch = BackTracking()), 
                      Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false, show_every=1))

 * Status: success

 * Candidate solution
    Final objective value:     4.393757e+04

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.39e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.59e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 5.61e-04 ≰ 1.0e-05

 * Work counters
    Seconds run:   7596  (vs limit Inf)
    Iterations:    2018
    f(x) calls:    2156
    ∇f(x) calls:   2019


In [60]:
theta_hat_mix_optim_ad = optimize(td, startvals, LBFGS(), 
Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false, show_every=50))

 * Status: failure (objective increased between iterations)

 * Candidate solution
    Final objective value:     4.393757e+04

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 7.18e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 8.26e-09 ≰ 0.0e+00
    |f(x) - f(x')|         = 7.28e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.66e-16 ≰ 0.0e+00
    |g(x)|                 = 3.24e-03 ≰ 1.0e-05

 * Work counters
    Seconds run:   6066  (vs limit Inf)
    Iterations:    727
    f(x) calls:    4417
    ∇f(x) calls:   4280


In [61]:
  theta_hat_mix_mle_ad = theta_hat_mix_optim_ad.minimizer
    println(theta_hat_mix_mle_ad)

[-0.04777877883533606, -0.47898270632061074, 1.3192068371897292, -0.007828243306700419, 0.3872993108319169, -2.3706206042816667, 0.10853115013838868, 0.7881421211966766, -4.830801965152722, 0.04888970119859712, 0.5506886770187716, -4.754547069771281, 0.028378170943471, -0.4743409826824806, -4.839365770733797, 0.12089211111352804, -0.588758745069359, -8.693494922428261, 0.10783378781742979, -0.5349464811147346, -5.843150991831915, 2.6928517144885613, 2.4322748489032886]


In [62]:
# # evaluate the Hessian at the estimates
# H  = Optim.hessian!(td, theta_hat_mix_optim_ad) # it bugs out at this point, and I don't now why
# # theta_hat_mix_mle_ad_se = sqrt.(diag(inv(H)))
# # println([theta_hat_mix_mle_ad theta_hat_mix_mle_ad_se]) # these standard errors match Stata

In [63]:
function mixlogit_MC_with_Z(theta, X, Z, y, R)
    
    alpha = theta[1:end-2]
    gamma = theta[end-1]
    sigma = exp(theta[end]) # makes sure sigma is never 0 (which would make the Normal density undefined)
    K = size(X,2)
    J = length(unique(y))
    N = length(y)
    bigY = zeros(N,J)
    for j=1:J
        bigY[:,j] = y.==j
    end
    bigAlpha = [reshape(alpha,K,J-1) zeros(K)]
    
    # now write P as a function of variable of integration
    T = promote_type(eltype(X),eltype(theta))
    function like_int(T,N,J,X,Z,bigAlpha,gamma,sigma,R)
        draws = 8*sigma.*Base.rand(R).-4*sigma
        out = zeros(T,N)
        for r=1:R
            num   = zeros(T,N,J)
            dem   = zeros(T,N)
            P     = zeros(T,N,J)
            for j=1:J
                num[:,j] = exp.(X*bigAlpha[:,j] .+ (Z[:,j] .- Z[:,J])*draws[r])
                dem .+= num[:,j]
            end
            P = num./repeat(dem,1,J)
            out .+= vec( (8*sigma/R)*( prod(P.^bigY; dims=2) )*pdf(Normal(gamma,sigma),draws[r]) ) # why do we have to use vec here? because vectors in Julia are "flat" and the vec operator flattens a one-dimensional array.
        end
        return out
    end
    
    intlike = like_int(T,N,J,X,Z,bigAlpha,gamma,sigma,R)
    loglike = -sum(log.(intlike))
    return loglike
end

mixlogit_MC_with_Z (generic function with 1 method)

In [64]:
td = TwiceDifferentiable(theta -> mixlogit_MC_with_Z(theta, X, Z, y, 100), startvals; autodiff = :forward)
    # run the optimizer
#theta_hat_mix_optim_ad = optimize(td, startvals, LBFGS(), Optim.Options(g_tol = 1e-5, iterations=2, show_trace=true, show_every=1))
theta_hat_mix_optim_ad = optimize(td, startvals, LBFGS(), Optim.Options(g_tol = 1e-5, iterations=100_000, show_trace=false, show_every=50))

theta_hat_mix_mle_ad = theta_hat_mix_optim_ad.minimizer
println(theta_hat_mix_mle_ad)

[-0.04326772506719707, -0.7357681536728756, 1.1618560316958133, -0.008345407858326291, 0.15008395109218325, -2.4388362734012152, 0.0981313084416861, 0.6507473825148847, -4.049617172377045, 0.052518879849848876, 0.27727260688726524, -4.340404524591591, 0.021648810078757155, -0.6880007065687087, -4.794184386063019, 0.11916101559881587, -0.9976439945194641, -7.914542311020034, 0.09652301391792456, -0.7818580714530835, -5.609416232680977, 4.5094358737850095, 2.295820123700223]


In [65]:
# evaluate the Hessian at the estimates
# H  = Optim.hessian!(td, theta_hat_mix_optim_ad) # it bugs out at this point, and I don't now why (it's possibly due to me making sigma piecewise above)
# theta_hat_mix_mle_ad_se = sqrt.(diag(inv(H)))
# println([theta_hat_mix_mle_ad theta_hat_mix_mle_ad_se]) # these standard errors match Stata